---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

#### **4. `LongContextReorder`**

#### **1) `긴 문맥 재정렬`**

* **긴 문맥 재정렬의 `필요성`**

  * 10개 이상 검색된 문서를 포함할 경우 성능 저하 *(모델의 아키텍처와 상관 X)*

  * 모델이 긴 컨텍스트 중간에 있는 관련 정보에 접근 시: 제공된 문서를 무시하는 경향이 있음 *(아래 관련 논문 참고)*

    * **`이 문제를 피하기 위해 검색 후 문서의 순서를 재배열하여 성능 저하를 방지할 수 있음`**

* *참고 - [`관련 논문`](https://arxiv.org/abs/2307.03172)*: https://arxiv.org/abs/2307.03172

---

#### **2) `설정`**

* **`Chroma`** 벡터 저장소 사용 → 텍스트 데이터를 저장하고 검색할 수 있는 **`retriever`** 생성

* **`retirever`** 의 **`invoke`** 메서드 사용 → 주어진 쿼리에 대해 관련성이 높은 문서 검색

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()                               # True

In [ ]:
from langsmith import Client
from langsmith import traceable

import os

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 셀 출력

    ```markdown
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

* **`try_1`**: 허깅페이스 384차원 모델

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_transformers import LongContextReorder
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# 1단계: Embeddings 사용
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
    )

embeddings = embeddings                                     # 11.3s 소요

In [ ]:
# 임베딩 모델 테스트

dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)                                                     # 384
print("✅ HuggingFaceEmbeddings 초기화 완료!")                               # ✅ HuggingFaceEmbeddings 초기화 완료!

In [ ]:
# 2단계: 샘플 문서 리스트

texts = [
    "이건 그냥 내가 아무렇게나 적어본 글입니다.",
    "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
    "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
    "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
    "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
    "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
    "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
    "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
    "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
    "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
]

# 테스트 출력
print(texts)

<small>

* 셀 출력

    ```python
    ['이건 그냥 내가 아무렇게나 적어본 글입니다.', '사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.', '아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.', '챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.', '챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.', '애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.', 'ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.', '비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.', 'ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.', 'FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.']
    ```

In [ ]:
# 3단계: 검색기 생성하기

retriever = Chroma.from_texts(
    texts,
    embedding=embeddings).as_retriever(
        search_kwargs={"k":10}                          # k = 10으로 설정
    )                                                   # 2.6s 소요                                              

* 검색기에 쿼리 입력 → 검색 수행

In [ ]:
query = "ChatGPT에 대해 무엇을 말해줄 수 있나요?"

# 4단계: 관련성 점수에 따라 정렬된 관련 문서 가져오기
docs = retriever.invoke(query)
docs

<small>

* 셀 출력 (0.1s)

    ```python
    [Document(metadata={}, page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.'),
    Document(metadata={}, page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
    Document(metadata={}, page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
    Document(metadata={}, page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'),
    Document(metadata={}, page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),
    Document(metadata={}, page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),
    Document(metadata={}, page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'),
    Document(metadata={}, page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),
    Document(metadata={}, page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'),
    Document(metadata={}, page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.')]
    ```

* **`LongContextReorder`** 클래스의 인스턴스인 **`reordering`** 생성하기

  * **`reordering.transform_documents(docs)`** 호출 → 문서 목록 **`docs`** 재정렬

  * 더 관련된 문서 = 시작과 끝에 위치하도록 재정렬
  * 덜 관련된 문서 = 목록의 중간에 위치

In [ ]:
# 5단계: 문서 재정렬하기

reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# 4개의 관련 문서가 시작과 끝에 위치하는지 확인해보기
reordered_docs

<small>

* 셀 출력

    ```python
    [Document(metadata={}, page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
    Document(metadata={}, page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'),                       # 관련성 ↓
    Document(metadata={}, page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),
    Document(metadata={}, page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),
    Document(metadata={}, page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.'),
    Document(metadata={}, page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'),    # 관련성 ↑
    Document(metadata={}, page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'),
    Document(metadata={}, page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),   # 관련성 ↓
    Document(metadata={}, page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
    Document(metadata={}, page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.')]
    ```

<small>

* **`교재 속 재정렬 정답`**

    ```python
    [Document(page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'), 
    Document(page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'), 
    Document(page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'), 
    Document(page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'), 
    Document(page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.'), 
    Document(page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'), 
    Document(page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'), 
    Document(page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'), 
    Document(page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'), 
    Document(page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.')]
    ```

<br>

* **`제대로 재정렬이 되지 않음`**
  * **`384차원 임베딩 모델`** 의 낮은 표현력
  * **`작은 차이의 유사도 점수 → 재정렬 효과 미미`**
  * 한국어 데이터에 대한 모델 성능 한계: 교재 속 `OpenAI` 임베딩 모델 = 영문 데이터에 익숙함

---

* **`try_2`**: 허깅페이스 768차원 모델

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_transformers import LongContextReorder
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# 1단계: Embeddings 사용
embeddings2 = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",   # 768차원 (2배!)
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

embeddings = embeddings2                                     # 11.2s 소요

In [ ]:
# 임베딩 모델 테스트

dimension_size = len(embeddings2.embed_query("hello world"))
print(dimension_size)                                                     # 768
print("✅ HuggingFaceEmbeddings2 초기화 완료!")                              # ✅ HuggingFaceEmbeddings2 초기화 완료!

In [ ]:
# 2단계: 샘플 문서 리스트

texts = [
    "이건 그냥 내가 아무렇게나 적어본 글입니다.",
    "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
    "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
    "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
    "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
    "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
    "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
    "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
    "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
    "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
]

In [ ]:
# 3단계: 검색기 생성하기

retriever = Chroma.from_texts(
    texts,
    embedding=embeddings2).as_retriever(
        search_kwargs={"k":10}                          # k = 10으로 설정
    )                                                   # 2.3s 소요   

In [ ]:
query = "ChatGPT에 대해 무엇을 말해줄 수 있나요?"

# 4단계: 관련성 점수에 따라 정렬된 관련 문서 가져오기
docs = retriever.invoke(query)
docs

<small>

* 셀 출력 (0.2s)

    ```python
    [Document(metadata={}, page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.'),
    Document(metadata={}, page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
    Document(metadata={}, page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
    Document(metadata={}, page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'),
    Document(metadata={}, page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),
    Document(metadata={}, page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),
    Document(metadata={}, page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),  # 관련성 ↑
    Document(metadata={}, page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'),
    Document(metadata={}, page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.'),   # 관련성 ↓
    Document(metadata={}, page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.')]
    ```

In [ ]:
# 관련성 점수에 따라서 출력해보기

print("🔍 기본 검색 결과 (관련성 순):")
for i, doc in enumerate(docs):
    print(f"[{i}] {doc.page_content}")

print("\n" + "="*60)

<small>

* 깔끔하게 출력해보기

    ```python
    🔍 기본 검색 결과 (관련성 순):
    [0] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
    [1] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.
    [2] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
    [3] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.
    [4] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
    [5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
    [6] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
    [7] 이건 그냥 내가 아무렇게나 적어본 글입니다.
    [8] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.
    [9] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.

    ============================================================
    ```

* **`LongContextReorder`** 클래스의 인스턴스인 **`reordering`** 생성하기

  * **`reordering.transform_documents(docs)`** 호출 → 문서 목록 **`docs`** 재정렬

  * 더 관련된 문서 = 시작과 끝에 위치하도록 재정렬
  * 덜 관련된 문서 = 목록의 중간에 위치

In [ ]:
# 5단계: 문서 재정렬하기

reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# 4개의 관련 문서가 시작과 끝에 위치하는지 확인해보기
reordered_docs

<small>

* 재정렬 결과

    ```python
    [Document(metadata={}, page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
    Document(metadata={}, page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'), # 관련성 ↓
    Document(metadata={}, page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),       # 관련성 ↓
    Document(metadata={}, page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'),                            # 관련성 ↓
    Document(metadata={}, page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'),        # 관련성 ↑
    Document(metadata={}, page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.'),
    Document(metadata={}, page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),
    Document(metadata={}, page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),           # 관련성 ↓
    Document(metadata={}, page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
    Document(metadata={}, page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.')]
    ```

<small>

* **`교재 속 재정렬 정답`**

    ```python
    [Document(page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'), 
    Document(page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'), 
    Document(page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'), 
    Document(page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'), 
    Document(page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.'), 
    Document(page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'), 
    Document(page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'), 
    Document(page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'), 
    Document(page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'), 
    Document(page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.')]
    ```

<br>

* 마찬가지로 관련성이 낮은 내용이 앞, 뒤에 정렬되어 있는 것을 발견

In [ ]:
# 정렬해서 출력해보기

print("🔄 재정렬 후 결과 (관련성 높은 문서가 앞뒤로):")
for i, doc in enumerate(reordered_docs):
    print(f"[{i}] {doc.page_content}")

<small>

* 깔끔하게 출력해보기

    ```python
    🔄 재정렬 후 결과 (관련성 높은 문서가 앞뒤로):
    [0] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.
    [1] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.
    [2] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
    [3] 이건 그냥 내가 아무렇게나 적어본 글입니다.
    [4] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
    [5] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.
    [6] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
    [7] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
    [8] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
    [9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
    ```

---

* **`Q. 낮은 차원 임베딩의 문제일까, 유사도 점수의 문제일까?`**

---

* **`try_3`**: 허깅페이스 `fake` 모델로 다양한 차원으로 시도해보기

In [ ]:
from langchain_core.embeddings import FakeEmbeddings
from langchain_community.document_transformers import LongContextReorder
from langchain_community.vectorstores import Chroma

def test_fake_embeddings_dimensions():
    """FakeEmbeddings 차원별 LongContextReorder 성능 테스트"""
    
    dimensions_to_test = [384, 768, 1024, 1536, 2048]

    
    texts = [
        "이건 그냥 내가 아무렇게나 적어본 글입니다.",
        "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
        "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
        "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
        "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
        "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
        "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
        "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
        "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
        "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
    ]
    
    query = "ChatGPT에 대해 무엇을 말해줄 수 있나요?"
    
    for dim in dimensions_to_test:
        print(f"\n🔬 {dim}차원 FakeEmbeddings 테스트:")
        print("="*50)
        
        # FakeEmbeddings 생성
        embeddings3 = FakeEmbeddings(size=dim)
        
        # 검색기 생성
        retriever = Chroma.from_texts(texts, embedding=embeddings3).as_retriever(
            search_kwargs={"k": len(texts)}
        )
        
        # 검색
        docs = retriever.invoke(query)
        
        print("기본 순서:")
        for i, doc in enumerate(docs):
            print(f"  [{i}] {doc.page_content[:30]}...")
        
        # LongContextReorder 적용
        reordering = LongContextReorder()
        reordered_docs = reordering.transform_documents(docs)
        
        print("재정렬 후:")
        for i, doc in enumerate(reordered_docs):
            print(f"  [{i}] {doc.page_content[:30]}...")
        
        # 개선도 측정
        original_order = [doc.page_content for doc in docs]
        reordered_order = [doc.page_content for doc in reordered_docs]
        
        if original_order != reordered_order:
            print("  ✅ 순서 변경됨!")
        else:
            print("  ❌ 순서 변경 없음")

In [ ]:
# 실행

test_fake_embeddings_dimensions()

<small>

* 차원별 출력 결과 보기

    ```markdown

    1. 🔬 384차원 FakeEmbeddings 테스트:
    ==================================================
    기본 순서:
        [0] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...           ⭕️ 관련성 ↑
        [1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...               ⭕️ 관련성 ↑
        [2] 이건 그냥 내가 아무렇게나 적어본 글입니다....                    # 관련성 ↓
        [3] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...              # 관련성 ↓
        [4] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...             # 관련성 ↑
        [5] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...              ❌ 관련성 ↑
        [6] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...              ❌ 관련성 ↑
        [7] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...            # 관련성 ↓
        [8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...               ⭕️ 관련성 ↑
        [9] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...            ❌ 관련성 ↓
    재정렬 후:
        [0] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...               ⭕️ 관련성 ↑
        [1] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...              ❌ 관련성 ↓
        [2] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...              # 관련성 ↑
        [3] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...            # 관련성 ↓
        [4] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...            # 관련성 ↓
        [5] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...               ❌ 관련성 ↑
        [6] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...              # 관련성 ↑
        [7] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...             ❌ 관련성 ↓
        [8] 이건 그냥 내가 아무렇게나 적어본 글입니다....                    ❌ 관련성 ↓
        [9] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...           ⭕️ 관련성 ↑
    ✅ 순서 변경됨!

    2. 🔬 768차원 FakeEmbeddings 테스트:
    ==================================================
    기본 순서:
        [0] 이건 그냥 내가 아무렇게나 적어본 글입니다....                   # 관련성 ↓
        [1] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...             # 관련성 ↓
        [2] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...           # 관련성 ↓            
        [3] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...             # 관련성 ↑
        [4] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...             # 관련성 ↑
        [5] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...          # 관련성 ↑
        [6] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...              # 관련성 ↑
        [7] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...           ❌ 관련성 ↓
        [8] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...              # 관련성 ↑
        [9] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...           ❌ 관련성 ↑
    재정렬 후:
        [0] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...             ❌ 관련성 ↓
        [1] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...             ❌ 관련성 ↑
        [2] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...          ❌ 관련성 ↑
        [3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...
        [4] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...
        [5] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...              ❌ 관련성 ↑
        [6] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...              ❌ 관련성 ↑
        [7] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...             ❌ 관련성 ↑
        [8] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...           ❌ 관련성 ↓
        [9] 이건 그냥 내가 아무렇게나 적어본 글입니다....                   ❌ 관련성 ↓
    ✅ 순서 변경됨!


    3. 🔬 1024차원 FakeEmbeddings 테스트:
    ==================================================
    기본 순서:
        [0] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...             # 관련성 ↑             
        [1] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...             ❌ 관련성 ↓
        [2] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...          # 관련성 ↑
        [3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...            # 관련성 ↓
        [4] 이건 그냥 내가 아무렇게나 적어본 글입니다....                   # 관련성 ↓
        [5] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...           # 관련성 ↓
        [6] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...              # 관련성 ↑
        [7] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...             # 관련성 ↑
        [8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...              # 관련성 ↑
        [9] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...           ❌ 관련성 ↓
    재정렬 후:
        [0] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...             ❌ 관련성 ↓
        [1] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...            ❌ 관련성 ↓
        [2] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...           ❌ 관련성 ↓
        [3] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...             ❌ 관련성 ↑
        [4] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...           ❌ 관련성 ↓
        [5] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...              ❌ 관련성 ↑
        [6] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...              ❌ 관련성 ↑
        [7] 이건 그냥 내가 아무렇게나 적어본 글입니다....                   ❌ 관련성 ↓
        [8] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...           ⭕️ 관련성 ↑
        [9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...             ⭕️ 관련성 ↑
    ✅ 순서 변경됨!


    4. 🔬 1536차원 FakeEmbeddings 테스트:
    ==================================================

    기본 순서:
        [0] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...             # 관련성 ↑ 
        [1] 이건 그냥 내가 아무렇게나 적어본 글입니다....                   ❌ 관련성 ↓
        [2] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...              # 관련성 ↑ 
        [3] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...           # 관련성 ↓
        [4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...             # 관련성 ↓
        [5] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...             ❌ 관련성 ↑
        [6] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...              ❌ 관련성 ↑
        [7] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...          ❌ 관련성 ↑
        [8] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...            ❌ 관련성 ↓
        [9] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...           ❌ 관련성 ↓
    재정렬 후:
        [0] 이건 그냥 내가 아무렇게나 적어본 글입니다....                   ❌ 관련성 ↓
        [1] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...           ❌ 관련성 ↓
        [2] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...             ❌ 관련성 ↑
        [3] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...          ❌ 관련성 ↑
        [4] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...           ❌ 관련성 ↓
        [5] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...            ❌ 관련성 ↓
        [6] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...              ❌ 관련성 ↑
        [7] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...             ❌ 관련성 ↓
        [8] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...              ⭕️ 관련성 ↑
        [9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...             ⭕️ 관련성 ↑
    ✅ 순서 변경됨!


    5. 🔬 2048차원 FakeEmbeddings 테스트:
    ==================================================
    기본 순서:
        [0] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...             # 관련성 ↓
        [1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...              # 관련성 ↑
        [2] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...              # 관련성 ↑
        [3] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...           # 관련성 ↓
        [4] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...             # 관련성 ↑
        [5] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...            # 관련성 ↓
        [6] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...             # 관련성 ↑
        [7] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...          # 관련성 ↑
        [8] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...           # 관련성 ↓
        [9] 이건 그냥 내가 아무렇게나 적어본 글입니다....                   # 관련성 ↓
    재정렬 후:
        [0] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로...              ⭕️ 관련성 ↑
        [1] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 ...           ❌ 관련성 ↓
        [2] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 ...            # 관련성 ↓
        [3] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성...          ❌ 관련성 ↑
        [4] 이건 그냥 내가 아무렇게나 적어본 글입니다....                   # 관련성 ↓
        [5] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으...           # 관련성 ↓
        [6] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이...             ❌ 관련성 ↑
        [7] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해...             ⭕️ 관련성 ↑
        [8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는...              ⭕️ 관련성 ↑
        [9] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에...             ❌ 관련성 ↓
    ✅ 순서 변경됨!

    ```

---

* **`try_4`**: 유사도 점수 기반으로 재정렬

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import FakeEmbeddings
import numpy as np
from typing import List
from langchain_core.documents import Document

embeddings4 = FakeEmbeddings(size=1536)

class CustomLongContextReorder:
    """Jay 전용 긴 문맥 재정렬기"""
    
    def __init__(self, embeddings4):
        self.embeddings = embeddings4
    
    def calculate_relevance_scores(self, query: str, docs: List[Document]) -> List[tuple]:
        """쿼리와 문서들 간의 관련성 점수 계산"""
        
        query_embedding = self.embeddings.embed_query(query)
        scores_with_docs = []
        
        for i, doc in enumerate(docs):
            doc_embedding = self.embeddings.embed_documents([doc.page_content])[0]
            
            # 코사인 유사도 계산
            similarity = np.dot(query_embedding, doc_embedding) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding)
            )
            
            scores_with_docs.append((similarity, i, doc))
            
        return scores_with_docs
    
    def reorder_documents(self, query: str, docs: List[Document]) -> List[Document]:
        """문서들을 관련성에 따라 재정렬 (높은 관련성: 앞뒤, 낮은 관련성: 중간)"""
        
        # 관련성 점수 계산
        scores_with_docs = self.calculate_relevance_scores(query, docs)
        
        # 점수 순으로 정렬 (높은 점수부터)
        scores_with_docs.sort(key=lambda x: x[0], reverse=True)
        
        print("📊 관련성 점수:")
        for score, idx, doc in scores_with_docs:
            print(f"  점수: {score:.4f} | {doc.page_content[:50]}...")
        
        # 재정렬: 높은 관련성 → 앞뒤, 낮은 관련성 → 중간
        reordered_docs = []
        n = len(docs)
        
        for i, (score, idx, doc) in enumerate(scores_with_docs):
            if i % 2 == 0:                                          # 짝수번째 (높은 관련성) → 앞쪽부터
                reordered_docs.insert(i // 2, doc)
            else:                                                   # 홀수번째 (낮은 관련성) → 뒤쪽부터
                reordered_docs.append(doc)
        
        # 더 정교한 재정렬: Sandwich 패턴
        if n >= 4:
            high_relevance = scores_with_docs[:n//2]                # 상위 절반
            low_relevance = scores_with_docs[n//2:]                 # 하위 절반
            
            sandwich_order = []
            
            # 높은 관련성 문서들을 앞뒤로 배치
            for i, (score, idx, doc) in enumerate(high_relevance):
                if i % 2 == 0:
                    sandwich_order.insert(0, doc)                   # 앞쪽
                else:
                    sandwich_order.append(doc)                      # 뒤쪽
            
            # 낮은 관련성 문서들을 중간에 배치
            mid_point = len(sandwich_order) // 2
            for score, idx, doc in low_relevance:
                sandwich_order.insert(mid_point, doc)
                mid_point += 1
            
            return sandwich_order
        
        return reordered_docs


In [ ]:
from langchain_core.embeddings import FakeEmbeddings

# 임베딩 모델
embeddings4 = FakeEmbeddings(size=1536)

# 🎬 사용 예시
def test_custom_reordering():
    """커스텀 재정렬 테스트"""

    # 임베딩 가져오기
    embeddings = embeddings4

    # 테스트 텍스트
    texts = [
        "이건 그냥 내가 아무렇게나 적어본 글입니다.",
        "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
        "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
        "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
        "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
        "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
        "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
        "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
        "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
        "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
    ]
    
    # 검색기 생성
    retriever = Chroma.from_texts(texts, embedding=embeddings4).as_retriever(
        search_kwargs={"k": 10}
    )
    
    query = "ChatGPT에 대해 무엇을 말해줄 수 있나요?"
    docs = retriever.invoke(query)
    
    print("🔍 기본 검색 결과:")
    for i, doc in enumerate(docs):
        print(f"[{i}] {doc.page_content}")
    
    # 커스텀 재정렬 적용
    reorderer = CustomLongContextReorder(embeddings)
    reordered_docs = reorderer.reorder_documents(query, docs)
    
    print(f"\n🔄 커스텀 재정렬 결과:")
    for i, doc in enumerate(reordered_docs):
        print(f"[{i}] {doc.page_content}")

In [ ]:
# 실행

test_custom_reordering()

<small>

* 커스텀 검색 결과

    ```markdown

    🔍 기본 검색 결과:
        [0] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
        [1] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
        [2] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.
        [3] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.
        [4] 이건 그냥 내가 아무렇게나 적어본 글입니다.
        [5] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
        [6] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
        [7] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.
        [8] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
        [9] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.

    📊 관련성 점수:
        점수: 0.0156  | 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니...
        점수: 0.0121  | 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다....
        점수: 0.0080  | FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다....
        점수: 0.0030  | 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다....
        점수: 0.0010  | 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다....
        점수: -0.0019 | ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다....
        점수: -0.0053 | 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다....
        점수: -0.0340 | ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있...
        점수: -0.0390 | 이건 그냥 내가 아무렇게나 적어본 글입니다....
        점수: -0.0513 | 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다....

    🔄 커스텀 재정렬 결과:
        [0] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.      ⭕️
        [1] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.             ❌
        [2] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.            ⭕️
        [3] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.        ❌
        [4] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.      ❌
        [5] 이건 그냥 내가 아무렇게나 적어본 글입니다.
        [6] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.                           ❌
        [7] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.      ⭕️
        [8] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.                       ❌
        [9] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.                        ⭕️

    ```

In [1]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import FakeEmbeddings
import numpy as np
from typing import List
from langchain_core.documents import Document

embeddings5 = FakeEmbeddings(size=1024)

class CustomLongContextReorder:
    """Jay 전용 긴 문맥 재정렬기"""
    
    def __init__(self, embeddings5):
        self.embeddings = embeddings5
    
    def calculate_relevance_scores(self, query: str, docs: List[Document]) -> List[tuple]:
        """쿼리와 문서들 간의 관련성 점수 계산"""
        
        query_embedding = self.embeddings.embed_query(query)
        scores_with_docs = []
        
        for i, doc in enumerate(docs):
            doc_embedding = self.embeddings.embed_documents([doc.page_content])[0]
            
            # 코사인 유사도 계산
            similarity = np.dot(query_embedding, doc_embedding) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding)
            )
            
            scores_with_docs.append((similarity, i, doc))
            
        return scores_with_docs
    
    def reorder_documents(self, query: str, docs: List[Document]) -> List[Document]:
        """문서들을 관련성에 따라 재정렬 (높은 관련성: 앞뒤, 낮은 관련성: 중간)"""
        
        # 관련성 점수 계산
        scores_with_docs = self.calculate_relevance_scores(query, docs)
        
        # 점수 순으로 정렬 (높은 점수부터)
        scores_with_docs.sort(key=lambda x: x[0], reverse=True)
        
        print("📊 관련성 점수:")
        for score, idx, doc in scores_with_docs:
            print(f"  점수: {score:.4f} | {doc.page_content[:50]}...")
        
        # 재정렬: 높은 관련성 → 앞뒤, 낮은 관련성 → 중간
        reordered_docs = []
        n = len(docs)
        
        for i, (score, idx, doc) in enumerate(scores_with_docs):
            if i % 2 == 0:                                          # 짝수번째 (높은 관련성) → 앞쪽부터
                reordered_docs.insert(i // 2, doc)
            else:                                                   # 홀수번째 (낮은 관련성) → 뒤쪽부터
                reordered_docs.append(doc)
        
        # 더 정교한 재정렬: Sandwich 패턴
        if n >= 4:
            high_relevance = scores_with_docs[:n//2]                # 상위 절반
            low_relevance = scores_with_docs[n//2:]                 # 하위 절반
            
            sandwich_order = []
            
            # 높은 관련성 문서들을 앞뒤로 배치
            for i, (score, idx, doc) in enumerate(high_relevance):
                if i % 2 == 0:
                    sandwich_order.insert(0, doc)                   # 앞쪽
                else:
                    sandwich_order.append(doc)                      # 뒤쪽
            
            # 낮은 관련성 문서들을 중간에 배치
            mid_point = len(sandwich_order) // 2
            for score, idx, doc in low_relevance:
                sandwich_order.insert(mid_point, doc)
                mid_point += 1
            
            return sandwich_order
        
        return reordered_docs


In [3]:
from langchain_core.embeddings import FakeEmbeddings

# 임베딩 모델
embeddings5 = FakeEmbeddings(size=1024)

# 🎬 사용 예시
def test_custom_reordering():
    """커스텀 재정렬 테스트"""

    # 임베딩 가져오기
    embeddings = embeddings5

    # 테스트 텍스트
    texts = [
        "이건 그냥 내가 아무렇게나 적어본 글입니다.",
        "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
        "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
        "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
        "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
        "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
        "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
        "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
        "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
        "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
    ]
    
    # 검색기 생성
    retriever = Chroma.from_texts(texts, embedding=embeddings5).as_retriever(
        search_kwargs={"k": 10}
    )
    
    query = "ChatGPT에 대해 무엇을 말해줄 수 있나요?"
    docs = retriever.invoke(query)
    
    print("🔍 기본 검색 결과:")
    for i, doc in enumerate(docs):
        print(f"[{i}] {doc.page_content}")
    
    # 커스텀 재정렬 적용
    reorderer = CustomLongContextReorder(embeddings)
    reordered_docs = reorderer.reorder_documents(query, docs)
    
    print(f"\n🔄 커스텀 재정렬 결과:")
    for i, doc in enumerate(reordered_docs):
        print(f"[{i}] {doc.page_content}")

In [ ]:
# 실행

test_custom_reordering()

<small>

* fake 1024 커스텀 결과

    ```markdown
    🔍 기본 검색 결과:
        [0] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
        [1] 자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다.
        [2] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.
        [3] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
        [4] 사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다.
        [5] OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다.
        [6] ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다.
        [7] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
        [8] 이건 그냥 내가 아무렇게나 적어본 글입니다.
        [9] 축구는 전 세계적으로 인기가 높은 스포츠입니다.

    📊 관련성 점수:
        점수: 0.0372  | 자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다....
        점수: 0.0210  | OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다....
        점수: 0.0202  | 이건 그냥 내가 아무렇게나 적어본 글입니다....
        점수: 0.0190  | FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다....
        점수: 0.0061  | ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있...
        점수: -0.0054 | 사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다....
        점수: -0.0158 | 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다....
        점수: -0.0185 | 축구는 전 세계적으로 인기가 높은 스포츠입니다....
        점수: -0.0411 | 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다....
        점수: -0.0762 | ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다....

    🔄 커스텀 재정렬 결과:
        [0] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
        [1] 이건 그냥 내가 아무렇게나 적어본 글입니다.
        [2] 사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다.
        [3] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
        [4] 축구는 전 세계적으로 인기가 높은 스포츠입니다.
        [5] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
        [6] ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다.
        [7] 자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다.
        [8] OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다.
        [9] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.
    ```

---

* **`try_5`**: 실제 유사도 기반 재정렬

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_transformers import LongContextReorder
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import FakeEmbeddings
import numpy as np
from typing import List
from langchain_core.documents import Document

embeddings4 = FakeEmbeddings(size=1536)

def create_working_long_context_reorder():
    """실제 동작하는 긴 문맥 재정렬 구현"""
    
    # 임베딩 가져오기
    embeddings = embeddings4
    
    # 더 극명한 차이의 텍스트 사용
    texts = [
        # ChatGPT 관련 (높은 관련성)
        "ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델입니다.",
        "사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다.",
        "ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다.",
        "OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다.",
        
        # 중간 관련성
        "인공지능 기술은 현대 사회에 많은 변화를 가져오고 있습니다.",
        "자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다.",
        
        # 낮은 관련성
        "애플은 아이폰과 맥북 등 다양한 전자제품을 생산하는 회사입니다.",
        "비트코인은 블록체인 기술을 기반으로 한 암호화폐입니다.",
        "축구는 전 세계적으로 인기가 높은 스포츠입니다.",
        "음식 배달 서비스는 코로나19 이후 급격히 성장했습니다.",
    ]
    
    # 검색기 생성
    retriever = Chroma.from_texts(texts, embedding=embeddings4).as_retriever(
        search_kwargs={"k": 10}
    )
    
    query = "ChatGPT에 대해 자세히 설명해주세요."
    
    # 기본 검색
    docs = retriever.invoke(query)
    
    print("🔍 기본 검색 결과 (유사도 순):")
    for i, doc in enumerate(docs):
        print(f"[{i}] {doc.page_content}")
    
    print("\n" + "="*60+ "\n")
    
    # LangChain LongContextReorder 적용
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    
    print("🔄 LongContextReorder 적용 후:")
    for i, doc in enumerate(reordered_docs):
        print(f"[{i}] {doc.page_content}")
    
    # 검증: 실제로 재정렬이 되었는지 확인
    if docs != reordered_docs:
        print("\n✅ 성공! 문서 순서가 재정렬되었습니다!\n")
        
        # 패턴 분석
        original_contents = [doc.page_content for doc in docs]
        reordered_contents = [doc.page_content for doc in reordered_docs]
        
        print("\n📊 재정렬 패턴 분석:")
        print("앞부분 (높은 관련성 예상):")
        for i in range(min(3, len(reordered_contents))):
            print(f"  {reordered_contents[i][:50]}...")
        
        print("뒷부분 (높은 관련성 예상):")
        for i in range(max(0, len(reordered_contents)-3), len(reordered_contents)):
            print(f"  {reordered_contents[i][:50]}...")
            
    else:
        print("\n❌ 재정렬되지 않음")

In [ ]:
# 실행

create_working_long_context_reorder()

<small>

* fake 1536 모델 결과

    ```markdown

    🔍 기본 검색 결과 (유사도 순):
        [0] 비트코인은 블록체인 기술을 기반으로 한 암호화폐입니다.
        [1] ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다.
        [2] 축구는 전 세계적으로 인기가 높은 스포츠입니다.
        [3] 애플은 아이폰과 맥북 등 다양한 전자제품을 생산하는 회사입니다.
        [4] 인공지능 기술은 현대 사회에 많은 변화를 가져오고 있습니다.
        [5] 자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다.
        [6] OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다.
        [7] 사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다.
        [8] ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델입니다.
        [9] 축구는 전 세계적으로 인기가 높은 스포츠입니다.

    ============================================================

    🔄 LongContextReorder 적용 후:
        [0] ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다.
        [1] 애플은 아이폰과 맥북 등 다양한 전자제품을 생산하는 회사입니다.
        [2] 자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다.
        [3] 사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다.
        [4] 축구는 전 세계적으로 인기가 높은 스포츠입니다.
        [5] ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델입니다.
        [6] OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다.
        [7] 인공지능 기술은 현대 사회에 많은 변화를 가져오고 있습니다.
        [8] 축구는 전 세계적으로 인기가 높은 스포츠입니다.
        [9] 비트코인은 블록체인 기술을 기반으로 한 암호화폐입니다.

    ✅ 성공! 문서 순서가 재정렬되었습니다!

    📊 재정렬 패턴 분석:
    - 앞부분 (높은 관련성 예상):
        ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다....
        애플은 아이폰과 맥북 등 다양한 전자제품을 생산하는 회사입니다....
        자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다....
    - 뒷부분 (높은 관련성 예상):
        인공지능 기술은 현대 사회에 많은 변화를 가져오고 있습니다....
        축구는 전 세계적으로 인기가 높은 스포츠입니다....
        비트코인은 블록체인 기술을 기반으로 한 암호화폐입니다....

    ```

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_transformers import LongContextReorder
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import FakeEmbeddings
import numpy as np
from typing import List
from langchain_core.documents import Document

embeddings5 = FakeEmbeddings(size=1024)

def create_working_long_context_reorder():
    """실제 동작하는 긴 문맥 재정렬 구현"""
    
    # 임베딩 가져오기
    embeddings = embeddings5
    
    # 더 극명한 차이의 텍스트 사용
    texts = [
        # ChatGPT 관련 (높은 관련성)
        "ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델입니다.",
        "사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다.",
        "ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다.",
        "OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다.",
        
        # 중간 관련성
        "인공지능 기술은 현대 사회에 많은 변화를 가져오고 있습니다.",
        "자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다.",
        
        # 낮은 관련성
        "애플은 아이폰과 맥북 등 다양한 전자제품을 생산하는 회사입니다.",
        "비트코인은 블록체인 기술을 기반으로 한 암호화폐입니다.",
        "축구는 전 세계적으로 인기가 높은 스포츠입니다.",
        "음식 배달 서비스는 코로나19 이후 급격히 성장했습니다.",
    ]
    
    # 검색기 생성
    retriever2 = Chroma.from_texts(texts, embedding=embeddings5).as_retriever(
        search_kwargs={"k": 10}
    )
    
    query = "ChatGPT에 대해 자세히 설명해주세요."
    
    # 기본 검색
    docs = retriever2.invoke(query)
    
    print("🔍 기본 검색 결과 (유사도 순):")
    for i, doc in enumerate(docs):
        print(f"[{i}] {doc.page_content}")
    
    print("\n" + "="*60+ "\n")
    
    # LangChain LongContextReorder 적용
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    
    print("🔄 LongContextReorder 적용 후:")
    for i, doc in enumerate(reordered_docs):
        print(f"[{i}] {doc.page_content}")
    
    # 검증: 실제로 재정렬이 되었는지 확인
    if docs != reordered_docs:
        print("\n✅ 성공! 문서 순서가 재정렬되었습니다!\n")
        
        # 패턴 분석
        original_contents = [doc.page_content for doc in docs]
        reordered_contents = [doc.page_content for doc in reordered_docs]
        
        print("\n📊 재정렬 패턴 분석:")
        print("앞부분 (높은 관련성 예상):")
        for i in range(min(3, len(reordered_contents))):
            print(f"  {reordered_contents[i][:50]}...")
        
        print("뒷부분 (높은 관련성 예상):")
        for i in range(max(0, len(reordered_contents)-3), len(reordered_contents)):
            print(f"  {reordered_contents[i][:50]}...")
            
    else:
        print("\n❌ 재정렬되지 않음")

In [ ]:
# 실행

create_working_long_context_reorder()

<small>

* fake 1024 모델

    ```markdown
    🔍 기본 검색 결과 (유사도 순):
        [0] 음식 배달 서비스는 코로나19 이후 급격히 성장했습니다.
        [1] ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델입니다.
        [2] OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다.
        [3] 사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다.
        [4] 비트코인은 블록체인 기술을 기반으로 한 암호화폐입니다.
        [5] ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다.
        [6] 축구는 전 세계적으로 인기가 높은 스포츠입니다.
        [7] 인공지능 기술은 현대 사회에 많은 변화를 가져오고 있습니다.
        [8] 자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다.
        [9] 애플은 아이폰과 맥북 등 다양한 전자제품을 생산하는 회사입니다.

    ============================================================

    🔄 LongContextReorder 적용 후:
        [0] ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델입니다.
        [1] 사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다.
        [2] ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다.
        [3] 인공지능 기술은 현대 사회에 많은 변화를 가져오고 있습니다.
        [4] 애플은 아이폰과 맥북 등 다양한 전자제품을 생산하는 회사입니다.
        [5] 자연어 처리 기술의 발전으로 인간과 컴퓨터의 소통이 향상되었습니다.
        [6] 축구는 전 세계적으로 인기가 높은 스포츠입니다.
        [7] 비트코인은 블록체인 기술을 기반으로 한 암호화폐입니다.
        [8] OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다.
        [9] 음식 배달 서비스는 코로나19 이후 급격히 성장했습니다.

    ✅ 성공! 문서 순서가 재정렬되었습니다!


    📊 재정렬 패턴 분석:
    - 앞부분 (높은 관련성 예상):
        ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델입니다....
        사용자와 자연스럽게 대화할 수 있는 ChatGPT는 혁신적인 AI 기술입니다....
        ChatGPT는 복잡한 질문에 대해서도 정확하고 유용한 답변을 제공합니다....
    - 뒷부분 (높은 관련성 예상):
        비트코인은 블록체인 기술을 기반으로 한 암호화폐입니다....
        OpenAI의 ChatGPT는 지속적인 학습을 통해 성능이 개선되고 있습니다....
        음식 배달 서비스는 코로나19 이후 급격히 성장했습니다....
    ```

In [9]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_transformers import LongContextReorder
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import FakeEmbeddings
import numpy as np
from typing import List
from langchain_core.documents import Document

embeddings5 = FakeEmbeddings(size=1024)

def create_working_long_context_reorder():
    """실제 동작하는 긴 문맥 재정렬 구현"""
    
    # 임베딩 가져오기
    embeddings = embeddings5
    
    # 더 극명한 차이의 텍스트 사용
    texts2 = [
        "이건 그냥 내가 아무렇게나 적어본 글입니다.",
        "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
        "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
        "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
        "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
        "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
        "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
        "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
        "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
        "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
]
    
    # 검색기 생성
    retriever2 = Chroma.from_texts(texts2, embedding=embeddings5).as_retriever(
        search_kwargs={"k": 10}
    )
    
    query = "ChatGPT에 대해 자세히 설명해주세요."
    
    # 기본 검색
    docs = retriever2.invoke(query)
    
    print("🔍 기본 검색 결과 (유사도 순):")
    for i, doc in enumerate(docs):
        print(f"[{i}] {doc.page_content}")
    
    print("\n" + "="*60+ "\n")
    
    # LangChain LongContextReorder 적용
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    
    print("🔄 LongContextReorder 적용 후:")
    for i, doc in enumerate(reordered_docs):
        print(f"[{i}] {doc.page_content}")
    
    # 검증: 실제로 재정렬이 되었는지 확인
    if docs != reordered_docs:
        print("\n✅ 성공! 문서 순서가 재정렬되었습니다!\n")
        
        # 패턴 분석
        original_contents = [doc.page_content for doc in docs]
        reordered_contents = [doc.page_content for doc in reordered_docs]
        
        print("\n📊 재정렬 패턴 분석:")
        print("앞부분 (높은 관련성 예상):")
        for i in range(min(3, len(reordered_contents))):
            print(f"  {reordered_contents[i][:50]}...")
        
        print("뒷부분 (높은 관련성 예상):")
        for i in range(max(0, len(reordered_contents)-3), len(reordered_contents)):
            print(f"  {reordered_contents[i][:50]}...")
            
    else:
        print("\n❌ 재정렬되지 않음")

In [ ]:
# 실행

create_working_long_context_reorder()

<small>

* `fake 1024` + 교재 속 `texts`

    ```markdown
    🔍 기본 검색 결과 (유사도 순):
        [0] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.
        [1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
        [2] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
        [3] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.
        [4] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
        [5] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
        [6] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
        [7] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
        [8] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
        [9] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.

    ============================================================

    🔄 LongContextReorder 적용 후:
        [0] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
        [1] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.
        [2] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
        [3] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
        [4] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
        [5] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
        [6] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
        [7] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
        [8] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
        [9] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.

    ✅ 성공! 문서 순서가 재정렬되었습니다!


    📊 재정렬 패턴 분석:
    - 앞부분 (높은 관련성 예상):
        챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다....
        FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다....
        ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있...
    - 뒷부분 (높은 관련성 예상):
        챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니...
        ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다....
        사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다....
    ```

---

#### **3) `Context Reordering`을 사용해 질의-응답 `체인` 생성하기**

In [12]:
# 임베딩 가져오기
embeddings = embeddings5
    
# 기존 텍스트 사용
texts2 = [
    "이건 그냥 내가 아무렇게나 적어본 글입니다.",
    "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
    "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
    "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
    "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
    "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
    "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
    "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
    "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
    "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
]
    
# 검색기 생성
retriever2 = Chroma.from_texts(texts2, embedding=embeddings5).as_retriever(
        search_kwargs={"k": 10}
    )
    
query = "ChatGPT에 대해 자세히 설명해주세요."
    
# 기본 검색
docs = retriever2.invoke(query)

In [13]:
def format_docs(docs):
    return "\n".join([doc.page_content for i, doc in enumerate(docs)])

In [ ]:
print(format_docs(docs))

<small>

* 셀 출력

    ```markdown
    ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
    ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
    ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
    아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
    ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
    ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
    애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
    ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
    FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.
    챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
    ```

In [15]:
def format_docs(docs):
    return "\n".join(
        [
            f"[{i}] {doc.page_content} [source: teddylee777@gmail.com]"
            for i, doc in enumerate(docs)
        ]
    )


def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    combined = format_docs(reordered_docs)
    print(combined)
    return combined

* 재정렬된 문서 출력해보기

In [ ]:
# 재정렬된 문서를 출력
_ = reorder_documents(docs)

<small>

* 재정렬된 문서 출력

    ```markdown
    [0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
    [1] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: teddylee777@gmail.com]
    [2] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
    [3] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: teddylee777@gmail.com]
    [4] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: teddylee777@gmail.com]
    [5] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: teddylee777@gmail.com]
    [6] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: teddylee777@gmail.com]
    [7] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
    [8] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: teddylee777@gmail.com]
    [9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: teddylee777@gmail.com]
    ```

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import FakeEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai
import os
import json


# 프롬프트 템플릿
template = """Given this text extracts:
{context}

-----
Please answer the following question:
{question}

Answer in the following languages: {language}
"""

# 프롬프트 정의
prompt = ChatPromptTemplate.from_template(template)

# LLM 정의
# Gemini 모델 초기화
gemini_lc = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

# Chain 정의
chain = (
    {
        "context": itemgetter("question")
        | retriever2
        | RunnableLambda(reorder_documents),                    # 질문을 기반으로 문맥 검색하기
        "question": itemgetter("question"),                     # 질문 추출하기
        "language": itemgetter("language"),                     # 답변 언어 추출하기
    }
    | prompt                                                    # 프롬프트 템플릿에 값을 전달하기
    | gemini_lc                                                 # 언어 모델에 프롬프트 전달하기
    | StrOutputParser()                                         # 모델의 출력을 문자열로 파싱하기
)

E0000 00:00:1759112756.510529 3043067 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


* **`question`** = 쿼리 입력, **`language`** = 언어 입력

* 재정렬된 문서의 검색 결과도 확인하기

In [ ]:
answer = chain.invoke(
    {"question": "ChatGPT에 대해 무엇을 말해줄 수 있나요?", "language": "KOREAN"}
)

<small>

* 셀 출력 (2.4s)

    ```markdown
    [0] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: teddylee777@gmail.com]
    [1] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
    [2] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: teddylee777@gmail.com]
    [3] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: teddylee777@gmail.com]
    [4] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
    [5] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: teddylee777@gmail.com]
    [6] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: teddylee777@gmail.com]
    [7] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: teddylee777@gmail.com]
    [8] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: teddylee777@gmail.com]
    [9] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
    ```

* 답변 출력해보기

In [ ]:
print(answer)

<small>

* 답변 출력

    ```markdown
    ChatGPT는 OpenAI에 의해 개발되었으며 지속적으로 개선되고 있습니다. 또한, 지속적인 학습과 업데이트를 통해 기능이 더욱 발전하고 있으며, 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
    ```

---

* **`gemini-2.5-flash`** 시도

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import FakeEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai
import os
import json


# 프롬프트 템플릿
template = """Given this text extracts:
{context}

-----
Please answer the following question:
{question}

Answer in the following languages: {language}
"""

# 프롬프트 정의
prompt = ChatPromptTemplate.from_template(template)

# LLM 정의
# Gemini 모델 초기화
gemini_lc2 = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Chain 정의
chain = (
    {
        "context": itemgetter("question")
        | retriever2
        | RunnableLambda(reorder_documents),                    # 질문을 기반으로 문맥 검색하기
        "question": itemgetter("question"),                     # 질문 추출하기
        "language": itemgetter("language"),                     # 답변 언어 추출하기
    }
    | prompt                                                    # 프롬프트 템플릿에 값을 전달하기
    | gemini_lc2                                                # 언어 모델에 프롬프트 전달하기
    | StrOutputParser()                                         # 모델의 출력을 문자열로 파싱하기
)

In [ ]:
answer = chain.invoke(
    {"question": "ChatGPT에 대해 무엇을 말해줄 수 있나요?", "language": "KOREAN"}
)

<small>

* `gemini-2.5-flash` 사용 (6.6s)

    ```markdown
    [0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
    [1] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: teddylee777@gmail.com]
    [2] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: teddylee777@gmail.com]
    [3] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
    [4] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: teddylee777@gmail.com]
    [5] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: teddylee777@gmail.com]
    [6] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: teddylee777@gmail.com]
    [7] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: teddylee777@gmail.com]
    [8] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: teddylee777@gmail.com]
    [9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: teddylee777@gmail.com]
    ```

In [ ]:
# 답변 출력해보기

print(answer)

<small>

* 답변 출력

    ```markdown
    ChatGPT는 OpenAI에 의해 개발되었으며, 지속적인 학습과 업데이트를 통해 그 기능이 발전하고 개선되고 있습니다. 또한, 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
    ```

* `LLM`모델보다 `LongContextReorder`의 품질이 더 중요한 영향을 미치는 것 같다고 판단됨

---

* *next: **`상위 문서 검색기 (ParentDocumentRetriever)`***

---